In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import json

## Parameters

In [2]:
class Config:
    # Embeddings
    word_embedding_dim = 300
    char_embedding_dim = 32
    pos_embedding_dim = 12
    
    # RNN
    hidden_size_word = 128
    hidden_size_char = 32
    l2_reg_lambda = 1e-5
    
    # Training parameters
    batch_size = 128
    num_epochs = 50
    display_every = 100
    evaluate_every = 250
    num_checkpoints = 5
    
    learning_rate = 1.0 
    decay_rate = 0.96
    
    # Testing parameters
    checkpoint_dir = ''
    
    UNK = "$UNK$"
    NUM = "$NUM$"
    NONE = "O"
    PAD = '$PAD$'

## Dataset 

ACE2005 dataset

In [3]:
import nltk
import re
import os

class Dataset:
    def __init__(self):
        self.all_tags, self.all_words, self.all_chars, self.all_entities = [], [], [], []
        self.dir_path = './data/ace2005'
        
    def processing_word(self, word):
        word = word.lower()
        if word.replace(',','').isdigit():
            word = Config.NUM
        return word
        
    def load_dataset(self, name):
        trigger_idx_col, words_col, tags_col, entities_col = [], [], [], []
        
        path = os.path.join(self.dir_path, '{}.json'.format(name))
        
        with open(path) as f:
            data = json.load(f)
            
            for item in data:
                words = item['tokens']
                
                words = []
                for word in item['tokens']:
                    word = self.processing_word(word)
                    
                    self.all_words.append(word)
                    self.all_chars.extend(list(word))
                    
                    words.append(word)
                    
                entities = [Config.NONE]*len(words)
                for entity_mention in item['golden_entity_mentions']:
                    for i in range(entity_mention['start'], entity_mention['end'] + 1):
                        if len(entities) <= i:
                                print('out of range! entity_mention:', entity_mention, words)
                                continue
                                
                        if i == entity_mention['start']:
                            entities[i] = 'B-{}'.format(entity_mention['entity_type'])
                        else:
                            entities[i] = 'I-{}'.format(entity_mention['entity_type'])
                            
                        self.all_entities.append(entities[i])
                
                
                for event_mention in item['golden_event_mentions']:
                    trigger_idx = event_mention['trigger']['start']
                    tags = [Config.NONE] * len(words)
                    
                    for argument in event_mention['arguments']:
                        for i in range(argument['start'], argument['end'] + 1):
                            if len(tags) <= i:
                                print('out of range! argument:', argument, words)
                                continue
                            
                            # "we collapse 8 of them that are time-related into one" (Yang and Mitchell, 2016)
                            if 'Time' in argument['role']:
                                argument['role'] = 'Time'
                            
                            if i == argument['start']:
                                tags[i] = 'B-{}'.format(argument['role'])
                            else:
                                tags[i] = 'I-{}'.format(argument['role'])
                            
                            self.all_tags.append(tags[i])
                    
                    trigger_idx_col.append(trigger_idx)
                    words_col.append(' '.join(words))
                    tags_col.append(' '.join(tags))
                    entities_col.append(' '.join(entities))
                
        return pd.DataFrame({'trigger_idx': trigger_idx_col, 
                             'words': words_col, 
                             'tags': tags_col,
                             'entities': entities_col})
        
    def load_datasets(self):
        train_df = self.load_dataset('train')
        dev_df =  self.load_dataset('dev')
        test_df = self.load_dataset('test')
        
        return train_df, dev_df, test_df

dataset = Dataset()
train_df, dev_df, test_df = dataset.load_datasets()

print('[Data Size]')
print('train : ', len(train_df))
print('dev : ', len(dev_df))
print('test.wsj : ', len(test_df))

train_df.head(10)

test_df.head(10)

out of range! entity_mention: {'end': 7, 'start': 6, 'entity_type': 'GPE:Nation', 'text': 'the country'} ['we', 'need', 'to', 'involve', 'the', 'u.n.', '.']
out of range! entity_mention: {'end': 7, 'start': 6, 'entity_type': 'GPE:Nation', 'text': 'the country'} ['we', 'need', 'to', 'involve', 'the', 'u.n.', '.']
out of range! entity_mention: {'end': 11, 'start': 6, 'entity_type': 'PER:Group', 'text': 'all Iraqi forces in the country'} ['we', 'need', 'to', 'involve', 'the', 'u.n.', '.']
out of range! entity_mention: {'end': 11, 'start': 6, 'entity_type': 'PER:Group', 'text': 'all Iraqi forces in the country'} ['we', 'need', 'to', 'involve', 'the', 'u.n.', '.']
out of range! entity_mention: {'end': 11, 'start': 6, 'entity_type': 'PER:Group', 'text': 'all Iraqi forces in the country'} ['we', 'need', 'to', 'involve', 'the', 'u.n.', '.']
out of range! entity_mention: {'end': 11, 'start': 6, 'entity_type': 'PER:Group', 'text': 'all Iraqi forces in the country'} ['we', 'need', 'to', 'involve'

out of range! entity_mention: {'end': 22, 'start': 19, 'entity_type': 'PER:Individual', 'text': 'Mickey Kaus of Slate'} ['the', 'golden', 'state', 'often', 'leads', 'the', 'way', 'for', 'the', 'nation', 'on', '``', 'reform', ',', "''", 'for', 'good', 'or', 'ill', '.']
out of range! entity_mention: {'end': 22, 'start': 19, 'entity_type': 'PER:Individual', 'text': 'Mickey Kaus of Slate'} ['the', 'golden', 'state', 'often', 'leads', 'the', 'way', 'for', 'the', 'nation', 'on', '``', 'reform', ',', "''", 'for', 'good', 'or', 'ill', '.']
out of range! entity_mention: {'end': 22, 'start': 19, 'entity_type': 'PER:Individual', 'text': 'Mickey Kaus of Slate'} ['the', 'golden', 'state', 'often', 'leads', 'the', 'way', 'for', 'the', 'nation', 'on', '``', 'reform', ',', "''", 'for', 'good', 'or', 'ill', '.']
out of range! entity_mention: {'end': 27, 'start': 19, 'entity_type': 'PER:Group', 'text': 'those who support or oppose the current redistricting process'} ['the', 'golden', 'state', 'often', '

,entities,tags,trigger_idx,words
0,B-GPE:Nation I-PER:Individual I-PER:Individual...,O O O O O O O O B-Time O B-Person I-Person I-P...,9,british chancellor of the exchequer gordon bro...
1,B-TIM:time I-PER:Individual I-PER:Individual I...,B-Person I-Person I-Person I-Person I-Person O...,5,former senior banker callum mccarthy begins wh...
2,B-TIM:time I-PER:Individual I-PER:Individual I...,B-Person I-Person I-Person I-Person I-Person O...,0,former senior banker callum mccarthy begins wh...
3,B-TIM:time I-PER:Individual I-PER:Individual I...,O O O O O O O O B-Position I-Position I-Positi...,26,former senior banker callum mccarthy begins wh...
4,B-PER:Individual O O O O B-Job-Title I-Job-Tit...,B-Person O O O O O O O O O O O O O O O O O O O...,2,davies is leaving to become chairman of the lo...
5,B-PER:Individual O O O O B-Job-Title I-Job-Tit...,B-Person O O O O B-Position I-Position I-Posit...,4,davies is leaving to become chairman of the lo...
6,O O O O O O O O B-ORG:Commercial I-ORG:Commerc...,O O O O O O O O B-Entity I-Entity O B-Entity O...,3,as well as previously holding senior positions...
7,O O O O O O O O B-ORG:Commercial I-ORG:Commerc...,O O O O O O O O O O O O O O O O B-Person O O O...,18,as well as previously holding senior positions...
8,B-PER:Individual I-PER:Individual I-PER:Indivi...,B-Entity I-Entity I-Entity I-Entity O O O B-En...,5,russian president vladimir putin 's summit wit...
9,B-PER:Individual I-PER:Individual I-PER:Indivi...,O O O O O O O O O O O O O O O O O O O O O O O ...,34,russian president vladimir putin 's summit wit...


In [4]:
word_list = list(set(dataset.all_words)) + [Config.PAD, Config.UNK]
word2idx = {w: i for i, w in enumerate(word_list)}
idx2word = {i: w for i, w in enumerate(word_list)}

tag_list = list(set(dataset.all_tags)) + [Config.NONE]
tag2idx = {w: i for i, w in enumerate(tag_list)}
idx2tag = {i: w for i, w in enumerate(tag_list)}

char_list = list(set(dataset.all_chars)) + [Config.PAD, Config.UNK]
char2idx = {w: i for i, w in enumerate(char_list)}
idx2char = {i: w for i, w in enumerate(char_list)}

entity_list = list(set(dataset.all_entities)) + [Config.PAD, Config.NONE, Config.UNK]
entity2idx = {w: i for i, w in enumerate(entity_list)}
idx2entity = {i: w for i, w in enumerate(entity_list)}

## Model

In [5]:
class Model:
    def __init__(self, 
               num_classes, 
               vocab_size, 
               char_size,
               word_embedding_dim, 
               char_embedding_dim,
               pos_embedding_dim,
               hidden_size_word,
               hidden_size_char):
        
        self.word_ids = tf.placeholder(tf.int32, shape=[None, None], name='word_ids') 
        self.entity_ids = tf.placeholder(tf.int32, shape=[None, None], name='entity_ids') 
        self.pos = tf.placeholder(tf.int32, shape=[None, None], name='pos')
        self.sequence_lengths = tf.placeholder(tf.int32, shape=[None], name="sequence_lengths")
        
        self.labels = tf.placeholder(tf.int32, shape=[None, None], name='labels')
        
        self.char_ids = tf.placeholder(tf.int32, shape=[None, None, None], name='char_ids') # [batch_size, max_sequence_length, max_word_length]
        self.word_lengths = tf.placeholder(tf.int32, shape=[None, None], name="word_lengths") # [batch_size, max_sequence_length]
        
        self.dropout = tf.placeholder(dtype=tf.float32, shape=[],name="dropout")
        
        initializer = tf.contrib.layers.xavier_initializer()
        
        # Word Embedding layer
        with tf.device('/cpu:0'), tf.variable_scope('word-embedding'):
            self._word_embeddings = tf.Variable(tf.random_uniform([vocab_size, word_embedding_dim], -0.25, 0.25), name='_word_embeddings')
            self.word_embeddings = tf.nn.embedding_lookup(self._word_embeddings, self.word_ids) # [batch_size, max_sequence_length, word_embedding_dim]
        
        with tf.variable_scope('position-embedding'):
            self._pos_embeddings = tf.get_variable('_pos_embeddings', [vocab_size, pos_embedding_dim], initializer=tf.keras.initializers.glorot_normal())
            self.pos_embeddings = tf.nn.embedding_lookup(self._pos_embeddings, self.pos)
            
        with tf.variable_scope('entity-embedding'):
            self._entity_embeddings = tf.get_variable('_entity_embeddings', [vocab_size, pos_embedding_dim], initializer=tf.keras.initializers.glorot_normal())
            self.entity_embeddings = tf.nn.embedding_lookup(self._entity_embeddings, self.pos)
            
        # Char Embedding Layer
        with tf.variable_scope('char-embedding'):
            self._char_embeddings = tf.get_variable(dtype=tf.float32, shape=[char_size, char_embedding_dim], name='_char_embeddings')
            
            # [batch_size, max_sequence_length, max_word_length, char_embedding_dim]
            self.char_embeddings = tf.nn.embedding_lookup(self._char_embeddings, self.char_ids) 
            
            s = tf.shape(self.char_embeddings)
            
            # [batch_size*max_sequence_length, max_word_length, char_embedding_dim]
            char_embeddings = tf.reshape(self.char_embeddings, shape=[s[0]*s[1], s[2], char_embedding_dim])
            word_lengths = tf.reshape(self.word_lengths, shape=[-1])
            
            fw_cell = tf.nn.rnn_cell.LSTMCell(hidden_size_char, state_is_tuple=True)
            bw_cell = tf.nn.rnn_cell.LSTMCell(hidden_size_char, state_is_tuple=True)
            
            _, ((_, output_fw), (_, output_bw)) = tf.nn.bidirectional_dynamic_rnn(cell_fw=fw_cell, 
                                                                                   cell_bw=bw_cell, 
                                                                                   inputs=char_embeddings,
                                                                                   sequence_length=word_lengths,
                                                                                   dtype=tf.float32)
            # shape: [batch_size*max_sequnce_length, 2*hidden_size_char]
            output = tf.concat([output_fw, output_bw], axis=-1)
            output = tf.reshape(output, shape=[s[0], s[1], 2*hidden_size_char])
            
            # shape: # [batch_size, max_sequence_length, word_embedding_dim + 2*hidden_size_char + pos_embedding_dim]
            self.word_embeddings = tf.concat([self.word_embeddings, output, self.pos_embeddings, self.entity_embeddings], axis=-1) 
            # self.word_embeddings = tf.nn.dropout(self.word_embeddings, self.dropout)
            
        # Bidirectional LSTM
        with tf.variable_scope("bi-lstm"):
            fw_cell = tf.nn.rnn_cell.LSTMCell(hidden_size_word)
            bw_cell = tf.nn.rnn_cell.LSTMCell(hidden_size_word)
            (output_fw, output_bw), _ = tf.nn.bidirectional_dynamic_rnn(cell_fw=fw_cell,
                                                                  cell_bw=bw_cell,
                                                                  inputs=self.word_embeddings,
                                                                  sequence_length= self.sequence_lengths, # [batch_size],
                                                                  dtype=tf.float32)
            
            self.rnn_outputs = tf.concat([output_fw, output_bw], axis=-1)  # [batch_size, max_sequence_length, 2*hidden_size_word]
            self.rnn_outputs = tf.nn.dropout(self.rnn_outputs, self.dropout)
        
        
        # Fully connected layer
        with tf.variable_scope('output'):
            self.W_output = tf.get_variable('W_output', shape=[2*hidden_size_word, num_classes],  dtype=tf.float32)
            self.b_output = tf.get_variable('b_output', shape=[num_classes], dtype=tf.float32, initializer=tf.zeros_initializer())
            
            nsteps = tf.shape(self.rnn_outputs)[1]
            rnn_outputs_flat = tf.reshape(self.rnn_outputs, [-1, 2*hidden_size_word])
            pred = tf.matmul(rnn_outputs_flat, self.W_output) + self.b_output
            
            self.logits = tf.reshape(pred, [-1, nsteps, num_classes]) # [batch_size, max_sequence_length, num_classes]
    
        # Calculate mean corss-entropy loss
        with tf.variable_scope('loss'):
            log_likelihood, trans_params = tf.contrib.crf.crf_log_likelihood(self.logits, self.labels, self.sequence_lengths)
            self.trans_params = trans_params  # need to evaluate it for decoding
            
            self.l2 = tf.add_n([tf.nn.l2_loss(v) for v in tf.trainable_variables()])
            self.loss = tf.reduce_mean(-log_likelihood) + Config.l2_reg_lambda * self.l2
            
#             When CRF is not in use
#             self.losses = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=self.logits, labels=self.labels)
#             mask = tf.sequence_mask(self.sequence_lengths)
#             losses = tf.boolean_mask(self.losses, mask)
#             self.loss = tf.reduce_mean(losses) 
        
    # Length of the sequence data
    @staticmethod
    def _length(seq):
        relevant = tf.sign(tf.abs(seq))
        length = tf.reduce_sum(relevant, reduction_indices=1)
        length = tf.cast(length, tf.int32)
        return length
    
    @staticmethod
    def viterbi_decode(logits, trans_params):
        # get tag scores and transition params of CRF
        viterbi_sequences = []

        # iterate over the sentences because no batching in vitervi_decode
        for logit, sequence_length in zip(logits, sequence_lengths):
            logit = logit[:sequence_length]  # keep only the valid steps
            viterbi_seq, viterbi_score = tf.contrib.crf.viterbi_decode(
                logit, trans_params)
            viterbi_sequences += [viterbi_seq]

        return np.array(viterbi_sequences)

## Function for train

In [6]:
# Pre-trained glove
def load_glove(word_embedding_dim, word2idx):
    download_path = tf.keras.utils.get_file(
      fname="glove.6B.zip", 
      origin="http://nlp.stanford.edu/data/glove.6B.zip", 
      extract=True)
    
    embedding_path = os.path.join(os.path.dirname(download_path), 'glove.6B.300d.txt')
    print('embedding_path :', embedding_path)

    # initial matrix with random uniform
    initW = np.random.randn(len(word2idx), word_embedding_dim).astype(np.float32) / np.sqrt(len(word2idx))
    # load any vectors from the glove
    print("Load glove file {0}".format(embedding_path))
    f = open(embedding_path, 'r', encoding='utf8')
    for line in f:
        splitLine = line.split(' ')
        word = splitLine[0]
        embedding = np.asarray(splitLine[1:], dtype='float32')
        if word in word2idx:
            initW[word2idx[word]] = embedding
    return initW

def batch_iter(df, batch_size, num_epochs, shuffle=True, tqdm_disable=False):
    """
    Generates a batch iterator for a dataset.
    """
    data_size = len(df)
    num_batches_per_epoch = int((data_size - 1) / batch_size) + 1
    for epoch in tqdm(range(num_epochs), disable=tqdm_disable):
        # Shuffle the data at each epoch
        if shuffle:
            shuffle_indices = np.random.permutation(np.arange(data_size))
            shuffled_df= df.iloc[shuffle_indices]
        else:
            shuffled_df = df
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, data_size)
            yield shuffled_df.iloc[start_index:end_index]    
            

def get_feed_dict(batch_df):
    max_length = max(map(lambda x : len(x.split(' ')), batch_df['words'].tolist()))
    
    max_length_word = 0
    for seq in batch_df['words'].tolist():
        for word in seq.split(' '):
            max_length_word = max(max_length_word, len(word))
    
    word_ids, sequence_lengths, labels, char_ids, entity_ids, word_lengths, pos = [], [], [], [], [], [], []
    for index, row in batch_df.iterrows():
        sentence = row['words'].split(' ')
        entities = row['entities'].split(' ')
        tags = row['tags'].split(' ')
        trigger_idx = row['trigger_idx']

        word_ids_row, labels_row, char_ids_row, entity_ids_row, word_lengths_row, pos_row = [], [], [], [], [], []
        for word in sentence:
            word_ids_row.append(word2idx[word])
        
            char_ids_row.append([char2idx[char] for char in word] + [char2idx[Config.PAD]]* (max_length_word - len(word)) )
            word_lengths_row.append(len(word))
            
        for entity in entities:
            entity_ids_row.append(entity2idx[entity])
            
        empty_char_ids = [char2idx[Config.PAD]]* max_length_word
        char_ids_row += [empty_char_ids] * (max_length - len(char_ids_row))
        word_lengths_row += [0] * (max_length - len(word_lengths_row))
        
        for tag in tags:
            labels_row.append(tag2idx[tag])

        if len(sentence) < max_length:
            word_ids_row += [word2idx[Config.PAD]]* (max_length - len(sentence))
            labels_row += [tag2idx[Config.NONE]]* (max_length - len(sentence))
            entity_ids_row += [entity2idx[Config.PAD]]* (max_length - len(sentence))
            
        for i in range(max_length):
            pos_row.append(i - trigger_idx)

        word_ids.append(word_ids_row)
        labels.append(labels_row)
        sequence_lengths.append(len(sentence))
        char_ids.append(char_ids_row)
        word_lengths.append(word_lengths_row)
        pos.append(pos_row)
        entity_ids.append(entity_ids_row)
    
    word_ids = np.array(word_ids)
    labels = np.array(labels)
    sequence_lengths = np.array(sequence_lengths)
    char_ids = np.array(char_ids)
    word_lengths = np.array(word_lengths)
    pos = np.array(pos)
    entity_ids = np.array(entity_ids)
    
    return word_ids, labels, sequence_lengths, char_ids, word_lengths, pos, entity_ids

def evaluation(y, preds, lengths):
    from sklearn.metrics import classification_report
    arg_answers, arg_preds = [], []
    
    accs = []
    correct_preds, total_correct, total_preds = 0.0, 0.0, 0.0
    for i in range(len(y)):
        sent_answers,sent_preds = [], []
        sent_answer_chunks, sent_pred_chunks = [], []
        
        for j in range(lengths[i]):
            sent_answers.append(idx2tag[y[i][j]])
            sent_preds.append(idx2tag[preds[i][j]])
            
            if idx2tag[y[i][j]] != Config.NONE:
                sent_answer_chunks.append(idx2tag[y[i][j]] + '-' + str(j))
            if idx2tag[preds[i][j]] != Config.NONE:
                sent_pred_chunks.append(idx2tag[preds[i][j]] + '-' + str(j))
    
        arg_answers.extend(sent_answers)
        arg_preds.extend(sent_preds)
        
        accs += [a == b for (a, b) in zip(sent_answers, sent_preds)]
        
        sent_answer_chunks = set(sent_answer_chunks)
        sent_pred_chunks = set(sent_pred_chunks)

        correct_preds += len(sent_answer_chunks & sent_pred_chunks)
        total_preds += len(sent_pred_chunks)
        total_correct += len(sent_answer_chunks)
    
    p = correct_preds / total_preds if correct_preds > 0 else 0
    r = correct_preds / total_correct if correct_preds > 0 else 0
    f1 = 2 * p * r / (p + r) if correct_preds > 0 else 0
    acc = np.mean(accs)
        
    print(classification_report(arg_answers, arg_preds))
    
    print('Tag based evaluation:\nprecision: {}, recall: {}, f1: {}'.format(p, r, f1))

## Train

In [7]:
import datetime
import time

from sklearn.metrics import f1_score
import warnings
import sklearn.exceptions
warnings.filterwarnings("ignore", category=sklearn.exceptions.UndefinedMetricWarning)

tf.reset_default_graph()
sess = tf.Session()
with sess.as_default():
    model = Model(
        num_classes=len(tag_list),
        vocab_size=len(word_list),
        char_size=len(char_list),
        word_embedding_dim=Config.word_embedding_dim,
        char_embedding_dim=Config.char_embedding_dim,
        pos_embedding_dim=Config.pos_embedding_dim,
        hidden_size_word=Config.hidden_size_word,
        hidden_size_char=Config.hidden_size_char
    )
    
    global_step = tf.Variable(0, name='global_step', trainable=False)
    # train_op = tf.train.AdamOptimizer(Config.learning_rate).minimize(model.loss, global_step=global_step)
    
    optimizer = tf.train.AdadeltaOptimizer(Config.learning_rate, Config.decay_rate, 1e-6)
    gvs = optimizer.compute_gradients(model.loss)
    capped_gvs = [(tf.clip_by_value(grad, -1.0, 1.0), var) for grad, var in gvs]
    train_op = optimizer.apply_gradients(capped_gvs, global_step=global_step)
    
    # Output directory for models and summary
    timestamp = str(int(time.time()))
    out_dir = os.path.abspath(os.path.join(os.path.curdir, "34.runs", timestamp))
    print("Writing to {}\n".format(out_dir))

    sess.run(tf.global_variables_initializer())

    pretrain_W = load_glove(Config.word_embedding_dim, word2idx)
    sess.run(model._word_embeddings.assign(pretrain_W))
    print("Success to load pre-trained glove model!\n")
    
    # Generate batches
    batches = batch_iter(train_df, Config.batch_size, Config.num_epochs)
    for batch_df in batches:
        word_ids, labels, sequence_lengths, char_ids, word_lengths, pos, entity_ids = get_feed_dict(batch_df)
        feed_dict = {
            model.word_ids: word_ids,
            model.labels: labels,
            model.sequence_lengths: sequence_lengths,
            model.char_ids: char_ids,
            model.word_lengths: word_lengths,
            model.dropout: 0.5,
            model.pos: pos,
            model.entity_ids: entity_ids
        }
        _, step, loss, logits, trans_params = sess.run([
            train_op, global_step, model.loss, model.logits, model.trans_params], feed_dict)
        
        predictions = model.viterbi_decode(logits, trans_params)
        
        # Training log display
        if step % Config.display_every == 0:
            time_str = datetime.datetime.now().isoformat()
            print("Train Evaluation {}: step {}, loss {:g}".format(time_str, step, loss))
#             evaluation(labels, predictions, sequence_lengths)
            
            
        # Evaluation
        if step % Config.evaluate_every == 0:
            batches = batch_iter(dev_df, Config.batch_size, 1, tqdm_disable=True)
            
            total_loss, predictions_all, labels_all, sequence_lengths_all  = 0, [], [], []
            for batch_df in batches:
                word_ids, labels, sequence_lengths, char_ids, word_lengths, pos, entity_ids = get_feed_dict(batch_df)
                feed_dict = {
                    model.word_ids: word_ids,
                    model.labels: labels,
                    model.sequence_lengths: sequence_lengths,
                    model.char_ids: char_ids,
                    model.word_lengths: word_lengths,
                    model.dropout: 1.0,
                    model.pos: pos,
                    model.entity_ids: entity_ids
                }
                loss, logits, trans_params = sess.run([model.loss, model.logits, model.trans_params], feed_dict)
                predictions = model.viterbi_decode(logits, trans_params)
                
                total_loss += loss
                predictions_all += predictions.tolist()
                labels_all += labels.tolist()
                sequence_lengths_all += sequence_lengths.tolist()
        
            time_str = datetime.datetime.now().isoformat()
            print("\nDev Evaluation\n{}: loss {:g}\n".format(time_str, total_loss/len(predictions_all)))
            evaluation(labels_all, predictions_all, sequence_lengths_all)


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


/home/seungwon/project/tf-notes/venv/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Writing to /home/seungwon/project/tf-notes/34.runs/1560538449

embedding_path : /home/seungwon/.keras/datasets/glove.6B.300d.txt
Load glove file /home/seungwon/.keras/datasets/glove.6B.300d.txt


  0%|          | 0/50 [00:00<?, ?it/s]

Success to load pre-trained glove model!

Train Evaluation 2019-06-15T03:54:59.700390: step 100, loss 26.283


...

Train Evaluation 2019-06-15T04:11:41.134671: step 9600, loss 3.13295


 98%|█████████▊| 49/50 [17:02<00:20, 20.68s/it]

Train Evaluation 2019-06-15T04:11:51.115023: step 9700, loss 3.10017

Dev Evaluation
2019-06-15T04:11:57.571004: loss 1.38214

               precision    recall  f1-score   support

B-Adjudicator       0.40      0.57      0.47         7
      B-Agent       0.62      0.43      0.51        37
   B-Artifact       0.60      0.51      0.55        57
   B-Attacker       0.41      0.30      0.34        74
B-Beneficiary       0.00      0.00      0.00         4
      B-Buyer       0.25      0.50      0.33         2
      B-Crime       0.44      0.44      0.44         9
  B-Defendant       0.50      0.44      0.47        18
B-Destination       0.73      0.73      0.73        41
     B-Entity       0.44      0.27      0.34        62
      B-Giver       0.56      0.18      0.27        28
 B-Instrument       0.41      0.36      0.38        36
      B-Money       0.80      0.50      0.62         8
        B-Org       0.00      0.00      0.00         2
     B-Origin       0.54      0.33      0.41   

100%|██████████| 50/50 [17:23<00:00, 20.90s/it]

Train Evaluation 2019-06-15T04:12:12.718787: step 9900, loss 2.89407


In [8]:
batches = batch_iter(test_df, Config.batch_size, 1, tqdm_disable=True)
total_loss, predictions_all, labels_all, sequence_lengths_all  = 0, [], [], []
for batch_df in batches:
    word_ids, labels, sequence_lengths, char_ids, word_lengths, pos, entity_ids = get_feed_dict(batch_df)
    feed_dict = {
        model.word_ids: word_ids,
        model.labels: labels,
        model.sequence_lengths: sequence_lengths,
        model.char_ids: char_ids,
        model.word_lengths: word_lengths,
        model.dropout: 1.0,
        model.pos: pos,
        model.entity_ids: entity_ids
    }
    loss, logits, trans_params = sess.run([model.loss, model.logits, model.trans_params], feed_dict)
    predictions = model.viterbi_decode(logits, trans_params)

    total_loss += loss
    predictions_all += predictions.tolist()
    labels_all += labels.tolist()
    sequence_lengths_all += sequence_lengths.tolist()

time_str = datetime.datetime.now().isoformat()
print("\Test Evaluation\n{}: loss {:g}\n".format(time_str, total_loss/len(predictions_all)))
evaluation(labels_all, predictions_all, sequence_lengths_all)

\Test Evaluation
2019-06-15T04:12:13.693613: loss 1.5683

               precision    recall  f1-score   support

B-Adjudicator       0.40      0.27      0.32        15
      B-Agent       0.40      0.19      0.26        32
   B-Artifact       0.49      0.44      0.47        63
   B-Attacker       0.48      0.32      0.38        31
B-Beneficiary       0.00      0.00      0.00         5
      B-Buyer       0.10      0.11      0.11        18
      B-Crime       0.81      0.68      0.74        38
  B-Defendant       0.58      0.58      0.58        33
B-Destination       0.53      0.66      0.59        38
     B-Entity       0.46      0.45      0.46       114
      B-Giver       0.14      0.09      0.11        11
 B-Instrument       0.67      0.33      0.44        12
      B-Money       0.56      0.42      0.48        12
        B-Org       0.38      0.30      0.33        20
     B-Origin       0.22      0.18      0.20        11
     B-Person       0.52      0.49      0.51        71
      